<a href="https://colab.research.google.com/github/BellKr/BellKr/blob/main/churn_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Final Model
ML1012 Krisanapong Wangtapaneeyakul

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Datasets/Telco-Customer-Churn.csv')
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


In [ ]:
df.columns


Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [ ]:
y=df['Churn']
y=y.replace({'Yes':1,'No':0}) #เปลี่ยน class ให้เป็น 0,1
y

0       0
1       0
2       1
3       0
4       1
       ..
7038    0
7039    0
7040    0
7041    1
7042    0
Name: Churn, Length: 7043, dtype: int64

# Feature selection

In [ ]:
X=df[[ 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure','MultipleLines',
      'Contract',
       'MonthlyCharges', ]] #, 'OnlineSecurity','OnlineBackup',  'PaymentMethod', , 'DeviceProtection','TechSupport','PhoneService', 'InternetService',  'StreamingTV', 'StreamingMovies',
X

,gender,SeniorCitizen,Partner,Dependents,tenure,MultipleLines,Contract,MonthlyCharges
0,Female,0,Yes,No,1,No phone service,Month-to-month,29.85
1,Male,0,No,No,34,No,One year,56.95
2,Male,0,No,No,2,No,Month-to-month,53.85
3,Male,0,No,No,45,No phone service,One year,42.30
4,Female,0,No,No,2,No,Month-to-month,70.70
...,...,...,...,...,...,...,...,...
7038,Male,0,Yes,Yes,24,Yes,One year,84.80
7039,Female,0,Yes,Yes,72,Yes,One year,103.20
7040,Female,0,Yes,Yes,11,No phone service,Month-to-month,29.60
7041,Male,1,Yes,No,4,Yes,Month-to-month,74.40


Training set and Test set

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y, random_state= 42)

In [ ]:
X_train.shape

(5282, 8)

Column Transformer

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [ ]:
transformer=make_column_transformer( (StandardScaler(),[ 'SeniorCitizen',  'tenure',
        'MonthlyCharges',]),(OneHotEncoder(),[ 'gender', 'Partner', 'Dependents',
       'Contract', 'MultipleLines']))

In [ ]:
X_train_transformed = transformer.fit_transform(X_train)
X_test_transformed = transformer.fit_transform(X_test)


#KNeighborsClassifier



In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score


In [ ]:
knn_9=KNeighborsClassifier(9)
knn_9.fit(X_train_transformed ,y_train) #ให้มันเรียนรู้ การสร้างโมเดลจากข้อมูล
result_knn9=cross_val_score(knn_9, X_train_transformed ,y_train,cv=5)
result_knn9.mean()

0.7726228104125454

In [ ]:
knn_15=KNeighborsClassifier(15)
knn_15.fit(X_train_transformed,y_train)
result_knn15=cross_val_score(knn_15, X_train_transformed,y_train,cv=5)
result_knn15.mean()

0.7728096958229409

In [ ]:
knn_27=KNeighborsClassifier(27)
knn_27.fit(X_train_transformed,y_train)
result_knn27=cross_val_score(knn_27, X_train_transformed,y_train,cv=5)
result_knn27.mean()

0.775082602276311

Train model


In [ ]:
knn27_final_model=KNeighborsClassifier(27)
knn27_final_model.fit(X_train_transformed,y_train)

KNeighborsClassifier(n_neighbors=27)

In [ ]:
knn27_final_model.score(X_train_transformed,y_train)

0.7925028398333964

In [ ]:
knn27_final_model.score(X_test_transformed,y_test)

0.7887563884156729

#GridsearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV
knn_grid_search=KNeighborsClassifier()

In [ ]:
paramerter_grid={'n_neighbors':(23,25,27,29,31,33,35,37,39,41,43,45,47)}

In [ ]:
grid_search_transformed =GridSearchCV(knn_grid_search,paramerter_grid,cv=5)

In [ ]:
grid_search_transformed .fit(X_train_transformed,y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': (23, 25, 27, 29, 31, 33, 35, 37, 39, 41,
                                         43, 45, 47)})

In [ ]:
grid_search_transformed.best_params_,grid_search_transformed.best_score_

({'n_neighbors': 35}, 0.7773560462716091)

In [ ]:
n_best_neighbors = grid_search_transformed.best_params_['n_neighbors']
n_best_neighbors

35

In [ ]:
grid_search_transformed.score(X_train_transformed,y_train)

0.7934494509655433

In [ ]:
grid_search_transformed.score(X_test_transformed,y_test)

0.7921635434412265

# LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logreg = LogisticRegression(max_iter=1000)

In [ ]:
logreg .fit(X_train_transformed, y_train)

LogisticRegression(max_iter=1000)

In [ ]:
logreg.score(X_train_transformed, y_train)

0.7934494509655433

In [ ]:
logreg.score(X_test_transformed, y_test)

0.8029528676888131

# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dtree =  DecisionTreeClassifier()

In [ ]:
dtree.fit(X_train_transformed, y_train)

DecisionTreeClassifier()

In [ ]:
dtree.score(X_test_transformed,y_test)

0.7058489494605338

In [ ]:
dtree.score(X_train_transformed,y_train)

0.9945096554335479

In [ ]:
dtree2 = DecisionTreeClassifier(min_samples_split= 30, max_depth= 10, min_samples_leaf=100)
dtree2.fit(X_train_transformed, y_train)

DecisionTreeClassifier(max_depth=10, min_samples_leaf=100, min_samples_split=30)

In [ ]:
dtree2.score(X_train_transformed, y_train)

0.7985611510791367

In [ ]:
dtree2.score(X_test_transformed, y_test)

0.787052810902896

#Naïve Bayes

In [ ]:
from sklearn.naive_bayes import BernoulliNB
bnb =  BernoulliNB()

In [ ]:
bnb.fit(X_train_transformed, y_train)

BernoulliNB()

In [ ]:
bnb.score(X_train_transformed, y_train)

0.7162059825823551

In [ ]:
bnb.score(X_test_transformed, y_test)

0.7279954571266326

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

In [ ]:
gnb.fit(X_train_transformed, y_train)

GaussianNB()

In [ ]:
gnb.score(X_train_transformed, y_train)

0.6940552820901174

In [ ]:
gnb.score(X_test_transformed, y_test)

0.696763202725724

#SVC

##Linear SVC

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
lin_svc = LinearSVC(max_iter= 6000) #ปรับค่า C ได้
lin_svc.fit(X_train_transformed, y_train)

LinearSVC(max_iter=6000)

In [ ]:
lin_svc.score(X_train_transformed, y_train)

0.7936387731919727

In [ ]:
lin_svc.score(X_test_transformed, y_test)

0.7995457126632595

In [ ]:
lin_svc.coef_

array([[ 0.04697342, -0.3517502 ,  0.31022097, -0.08626364, -0.09777929,
        -0.09746261, -0.08658032, -0.04561338, -0.13842955,  0.18509238,
        -0.11779126, -0.25134404, -0.20190227,  0.11791929, -0.10005995]])

In [ ]:
lin_svc.intercept_

array([-0.18404293])

In [ ]:
lin_svc.decision_function(X_test_transformed[:5])

array([-0.09043837, -0.89424182, -1.72020781,  0.11223773, -1.53999886])

##Poly SVC

In [ ]:
from sklearn.svm import SVC

In [ ]:
svc_poly =SVC(kernel ='poly') #สุดท้ายต้องหา gridseaarch ที่ดีที่สุด

In [ ]:
svc_poly.fit(X_train_transformed, y_train)

SVC(kernel='poly')

In [ ]:
svc_poly.score(X_train_transformed, y_train)

0.8023475956077244

In [ ]:
svc_poly.score(X_test_transformed, y_test)

0.794434980124929

## RBF SVC

In [ ]:
svc_rbf = SVC(kernel= 'rbf')

In [ ]:
svc_rbf.fit(X_train_transformed, y_train)

SVC()

In [ ]:
svc_rbf.score(X_train_transformed, y_train)

0.7983718288527073

In [ ]:
svc_rbf.score(X_test_transformed, y_test)

0.8001135718341851

#Voting Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import GaussianNB

In [ ]:
voting = VotingClassifier( estimators= [('knn', KNeighborsClassifier(n_best_neighbors)), ('dtree' , DecisionTreeClassifier(min_samples_split= 30, max_depth= 10, min_samples_leaf=100)), ('logit', LogisticRegression(max_iter=1000)),('svm', SVC(kernel ='rbf')), ('gnb', GaussianNB())])

In [ ]:
voting.fit(X_train_transformed, y_train)

VotingClassifier(estimators=[('knn', KNeighborsClassifier(n_neighbors=35)),
                             ('dtree',
                              DecisionTreeClassifier(max_depth=10,
                                                     min_samples_leaf=100,
                                                     min_samples_split=30)),
                             ('logit', LogisticRegression(max_iter=1000)),
                             ('svm', SVC()), ('gnb', GaussianNB())])

In [ ]:
voting.score(X_train_transformed, y_train)

0.8012116622491481

In [ ]:
voting.score(X_test_transformed, y_test)

0.7978421351504826

#Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier

In [ ]:
bag=  BaggingClassifier( KNeighborsClassifier(n_best_neighbors), max_samples= 1000, n_estimators=100, oob_score= True) #ในนี้เราลองใช้ KNeighborsClassifier ใน Bagging

In [ ]:
bag.fit(X_train_transformed, y_train)

BaggingClassifier(base_estimator=KNeighborsClassifier(n_neighbors=35),
                  max_samples=1000, n_estimators=100, oob_score=True)

In [ ]:
bag.oob_score_

0.7741385838697463

In [ ]:
bag.score(X_train_transformed, y_train)

0.777925028398334

In [ ]:
bag.score(X_test_transformed, y_test)

0.7802385008517888

#RandomForestClassifier


In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
forest =  RandomForestClassifier(n_estimators= 1000, max_depth=2, oob_score= True)

In [ ]:
forest.fit(X_train_transformed, y_train)

RandomForestClassifier(max_depth=2, n_estimators=1000, oob_score=True)

In [ ]:
forest.oob_score_

0.7368421052631579

In [ ]:
forest.score(X_train_transformed, y_train)

0.7368421052631579

In [ ]:
forest.score(X_test_transformed, y_test)

0.7279954571266326

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
parameter_grid = {'n_estimators': [100, 500, 1000], 'max_depth': [1,2,3,4,5]}

In [ ]:
grid_search = GridSearchCV( RandomForestClassifier(), parameter_grid, cv= 5)

In [ ]:
grid_search.fit(X_train_transformed, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [1, 2, 3, 4, 5],
                         'n_estimators': [100, 500, 1000]})

In [ ]:
grid_search.best_params_,grid_search.best_score_

({'max_depth': 5, 'n_estimators': 100}, 0.7898521042974685)

In [ ]:
grid_search.score(X_train_transformed, y_train)

0.7962892843619841

In [ ]:
grid_search.score(X_test_transformed, y_test)

0.787052810902896

#AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
ada = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators= 300, learning_rate= 0.5) #ปรับจูนได้

In [ ]:
ada.fit(X_train_transformed, y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=300)

In [ ]:
ada.score(X_train_transformed, y_train)

0.7968572510412723

In [ ]:
ada.score(X_test_transformed, y_test)

0.7910278250993753

#Gradient boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
gbc=GradientBoostingClassifier()

In [ ]:
gbc.fit(X_train_transformed, y_train)

GradientBoostingClassifier()

In [ ]:
gbc.score(X_train_transformed, y_train)

0.815032184778493

In [ ]:
gbc.score(X_test_transformed, y_test)

0.7898921067575241

#Stacking

In [ ]:
from sklearn.ensemble import StackingClassifier

In [ ]:
stacking= StackingClassifier( estimators= [('knn', KNeighborsClassifier(n_best_neighbors)), ('dtree' , DecisionTreeClassifier(min_samples_split= 30, max_depth= 10, min_samples_leaf=100)), ('logit', LogisticRegression(max_iter=1000)),('svm', SVC(kernel ='rbf')), ('gnb', GaussianNB())],final_estimator=RandomForestClassifier(), cv=5)

In [ ]:
stacking.fit(X_train_transformed, y_train)

StackingClassifier(cv=5,
                   estimators=[('knn', KNeighborsClassifier(n_neighbors=35)),
                               ('dtree',
                                DecisionTreeClassifier(max_depth=10,
                                                       min_samples_leaf=100,
                                                       min_samples_split=30)),
                               ('logit', LogisticRegression(max_iter=1000)),
                               ('svm', SVC()), ('gnb', GaussianNB())],
                   final_estimator=RandomForestClassifier())

In [ ]:
stacking.score(X_train_transformed, y_train)

0.8195759182127982

In [ ]:
stacking.score(X_test_transformed, y_test)

0.7881885292447472